# MLiP Neuronale Netze am Beispiel Kartontiefziehen
Kurs Maschinelles Lernen in der Produktion

### In diesem Notebook wird ein Entscheidungsbaum auf das Anwendungsbeispiel Kartontiefziehen trainiert.

Tiefziehen ist ein weit verbreitetes Umformverfahren. Bei diesem Beispiel wird jedoch kein Halbzeug aus Metall sondern aus Kartonage umgeformt. Die Besonderheit dabei ist, das Kartonage ein Fasermaterial ist und somit ein richtungsabhängiges Umformverhalten aufweist. Dadurch entstehen beim Umformen Falten. Jedoch ist bei diesem Prozess das Ziel einen möglichen ebenen und gleichmäßigen Flansch zu erhalten. 

Im Datensatz "Kartontiefziehen" werdne mehrere Eingangsparameter des Umformprozesses, wie bspw. Faltenhalterkraft und Temperaturen variert. Ziel ist es ein Vorhersagemodell zu entwickeln, mit dem man die maximale Höhe der Falten vorhersagen kann.  

### Data-Mining-Prozess:

![alt text](Prozess_Modellentwicklung_v2.png "Title")

### 0. Bibliotheken importieren

In [ ]:
# Importiere benötigte Bibliotheken
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV
import time

#Einstellungen für die Grafikausgabe
style = 'seaborn-whitegrid'
plt.style.use(style)
plt.rcParams.update({'font.size': 14})  # Schriftgröße aller Textzeichen im Graphen

TODO:
* Wähle eine Zahl zwischen 1 und 100 für die Generierung deiner spezifischen Zufallszahlen my_seed=

(Wähle für alle Notebooks in allen Übungen immer die gleiche Zahl (z.B. den Tag deines Geburtstags), dann sind die Ergebnisse der verschiedenen Machine-Learning-Verfahren vergleichbar da dann alle Notebooks mit der "gleichen" Folge an Zufallszahlen arbeiten)

AUSGABE:
* Gewählte Zufallszahl

In [ ]:
# Erstelle eigene Zufallszahlen
my_seed = TODO

# Ausgabe gewählte Zufallszahlen
print("\nGewählte Zahl für Zufallszahlen: \t" + str(my_seed))

### 1. Daten erfassen - Daten importieren

Import der Daten mittels der read_csv-Funktion von Pandas.  

In [ ]:
# Datensatz importieren
df = pd.read_csv('Kartontiefziehen.csv')

### 2.1 Daten erkunden
Infos zu den Daten:
* f_fh,	Faltenhalterkraft in N
* s_0,	Anfangsstärke Karton in mm
* t_fal,	Faltehaltertemperatur in °C
* t_stem,	Stempeltemperatur in °C
* t_zb,	Ziehbüchsentemperatur in °C
* f_punch_max,	maximale Stempelkraft
* thick_red_max,	maximale Kartonausdünnung am Ende der Simulation in % der Anfangsstärke	
* flx,	Flanscheinzug in X-Richtung in mm
* fly,	Flanscheinzug in Y-Richtung in mm
* h_falten,	Faltenhöhe in mm

In [ ]:
# Datensatz anzeigen
df.head(10)

In [ ]:
# Datensatz beschreiben
df.describe()

In [ ]:
# Zeige Streudiagramm aller Variablenkombinationen an
from pandas.plotting import scatter_matrix

scatter_matrix(df, alpha=0.2, figsize=(16, 16))
plt.show()

### 3.1 Daten vorbereiten - Daten aufteilen
Der Datensatz wird in Trainings- und Testdaten aufgeteilt.  
__Achtung__, die Trainingsdaten werden automatisch im Schritt GridSearch nochmals in Trainings- und Validationsdaten aufgeteilt.  

TODO:
- Lege den Anteil der Trainingsdaten mittels train_size fest  

Ausgabe:  
- Anzahl Trainings- und Testdaten

In [ ]:
# Daten in Trainings- und Test aufteilen

# Verhältnis Trainings- und Testdaten
train_size = TODO

# Aufteilung in Trainings- und Testdaten
X_train_, X_test_, y_train, y_test = train_test_split(df.drop(columns=['h_falten']),
                                                      df['h_falten'], 
                                                      train_size=train_size, 
                                                      random_state=my_seed)

# Ausgabe Datensätze und Anzahl Datenpunkte
print("\nAnzahl Traingsdaten: \t\t" + str(len(y_train)) + " / " + str(len(df)))
print("Anzahl Testdaten: \t\t" + str(len(y_test)) + " / " + str(len(df)))

### 3.2 Daten vorbereiten - Daten normieren
Die Normierung der Daten wird bei neuronalen Netzen stark empfohlen, weil es dem Netz das lernen erleichtert.  

Es stehen grundsätzlich der StandardScaler, der MinMaxScaler und der MaxAbsScaler zur Verfügung. Für dieses Beispiel wurde der StandardScaler vorbereitet.

Es können auch andere Scaler verwendet werden, die müssen entsprechend importiert werden und der Befehl scaler = ... muss angepasst werden. 

In [ ]:
# Skalierung der Daten mit dem StandardScaler
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X_train = pd.DataFrame(scaler.fit_transform(X_train_.values), index=X_train_.index, columns=X_train_.columns)
X_test = pd.DataFrame(scaler.transform(X_test_.values), index=X_test_.index, columns=X_test_.columns)

### 4.1 Modelle bilden - Modell importieren
Zuerst müssen wir das SVM Modell importieren, damit wir es später nutzen können. 

In [ ]:
# Importieren des Modells
from sklearn.neural_network import MLPRegressor

### 4.2 Modelle bilden - Optimale Hyperparameter mittels Gittersuche bestimmen

Um die optimalen Hyperparameter für das Neuronale Netz zu finden, wird eine __Gittersuche__ durchgeführt:

TODO:
- Schreibe die zu variierenden Hyperparameter listenweise in ein Dictionary 
- Die Struktur eines Neuronalen Netzes (Anzahl an hidden layers sowie Neuronen je hidden layer) wird wie folgt angegeben:
    - hidden_layers_sizes = (3,) -> 1 hidden layer mit 3 Neuronen
    - hidden_layers_sizes = (3,2) -> 2 hidden layers mit 3 und 2 Neuronen
    - hidden_layers_sizes = (5,5,2) -> 3 hidden layers mit 5, 5 und 3 Neuronen
    - Zur Erinnerung: Die input layer hat soviele Neuronen wie Inputparameter, die output layer hat bei der Regression 1 Neuron  
    
__Achtung__, es wurde fest der solver lbfgs (nicht zwingend beste Möglichkeit),\
 als random state der seed: random_state=my_seed\
 und die maximale Anzahl der Iterationen (max_iter) angepasst.\
 Diese Werte müssen nicht variert werden und wurden direkt dem Modell gegeben. 

In [ ]:
# Definition der Hyperparamter für die Gittersuche
hyper_parameters = {'hidden_layer_sizes': [TODO], 
                     TODO
                    }     

# Erstellung des MLP Modells
model = MLPRegressor(max_iter=50000, solver='lbfgs', random_state=my_seed)

### 4.3 Modelle bilden - Modelle per Gittersuche trainieren
Durchführen der Gittersuche  

AUSGABE:
* Anzahl der getesteten Hyperparameterkombinationen
* Zeitdauer für Gittersuche
* bestes Modell

In [ ]:
# Durchführung der Gittersuche
start_timer = time.monotonic()
gridSearch = GridSearchCV(model, hyper_parameters, return_train_score=True, scoring='neg_mean_absolute_error', cv=5, n_jobs=-2)
gridSearch.fit(X_train, y_train)

print("\nDie Gittersuche (" + str(len(pd.DataFrame(gridSearch.cv_results_)))
      + " Kombinationen) hat " + str("%.1f" % (time.monotonic() - start_timer))
      + " Sekunden gedauert.")

### 5.1 Modelle validieren - GridSearch Ergebnisse begutachten
In der Variablen GridSearch sind nun die Ergebnisse der Gittersuche gespeichert.  

Mit dem Befahl GridSearch.cv_results_ bekommen wir die Ergebnis-Tabelle der Gittersuche (hier: besten 5 Ergebnisse). 

TODO:
- Vergleiche die Trainings- und Testergebenis für die verschiedenen Splits, um sicherzustellen, dass ein gutes Modell gefunden wurde und kein Overfitting vorliegt.

In [ ]:
# Top 5 Ergebnisse
pd.set_option('display.max_columns', None)
pd.DataFrame(gridSearch.cv_results_).sort_values("mean_test_score", ascending=False).head(5)

### 5.2 Modelle validieren - Modell auswählen 
Ausgabe der besten Hyperparameter der Gittersuche

In [ ]:
# Beste Kombination der Hyperparameter
gridSearch.best_params_

Extraktion des besten Modells aus der Gittersuche.  
__Achtung__, dieses Modell wurde bereits mit allen Trainingsdaten trainiert. 

In [ ]:
# Extraktion des Modells
model = gridSearch.best_estimator_

### 5.3 Modelle validieren - Bewertung des Trainings
Bewertung des Trainings mittels dem MAE. 

TODO:
- Schreibe den Code für den MAE des Modells auf den Trainingsdaten
- Hinweis: Import der Funktion nicht vergessen

AUSGABE:
- MAE auf den Trainingsdaten

In [ ]:
# Vorhersage der Trainingsdaten mit Modell
y_train_pred = TODO

# Berechnung des MAE
print('MAE für die Trainingsdaten:')
TODO

Visualisierung der Modell-Vorhersage über den realen Werten.

In [ ]:
# Ausgabe Graph
min_train, max_train = y_train.min(), y_train.max()
plt.figure(figsize=(9, 8))
plt.plot(y_train, np.squeeze(y_train_pred), "o", alpha=0.4)
plt.plot([min_train, max_train], [min_train, max_train], "--", c=(0, 0, 0))
plt.xlabel("reale Faltenhöhe")
plt.ylabel("vorhergesagte Faltenhöhe")
plt.title("Vorzugsmodell - Trainingsdaten")
plt.show()

### 6.1 Modell testen & anwenden - Bewertung anhand der Testdaten

Um die Qualität/Güte des Modells zu bestimmen, berechnen wir nun den MAE für die Testdaten.

In [ ]:
# Vorhersage mit Modell anhand den Testdaten
y_test_pred = TODO

# Berechnug des MAE für die Testdaten
print('MAE für die Testdaten:')
TODO

Visualisierung der Modell-Vorhersage über den realen Werten.

Achtung:  
Falls das Ergebnis etwas überrascht, gerne die Auflösung ansehen, dort gibt es eine Erklärung dazu.

In [ ]:
# Ausgabe Graph
min_test, max_test = y_test.min(), y_test.max()
fig = plt.figure(figsize=(9, 8))
plt.plot(y_test, y_test_pred, "o", alpha=0.4)
plt.plot([min_test, max_test], [min_test, max_test], "--", c=(0, 0, 0))
plt.xlabel("reale Faltenhöhe")
plt.ylabel("vorhergesagte Faltenhöhe")
plt.title("Vorzugsmodell - Testdaten")
plt.show()